In [ ]:
# ==========================
# 1. 라이브러리 & 데이터 로드
# ==========================
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv("./data/proc/daily_raw_clean.csv")

# 심볼별, 날짜 오름차순 정렬
df = df.sort_values(["symbol", "date"]).reset_index(drop=True)

# ==========================
# 2. 가격/추세 지표
# ==========================
def add_price_trend_features(g):
    g = g.copy()
    
    # 이동평균
    g["ma_5"] = g["close"].rolling(5).mean()
    g["ma_10"] = g["close"].rolling(10).mean()
    g["ma_20"] = g["close"].rolling(20).mean()
    
    # 괴리율
    g["dist_ma5"] = g["close"] / g["ma_5"] - 1
    g["dist_ma10"] = g["close"] / g["ma_10"] - 1
    g["dist_ma20"] = g["close"] / g["ma_20"] - 1
    
    # 기울기
    g["ma10_slope"] = g["ma_10"] - g["ma_10"].shift(1)
    g["ma20_slope"] = g["ma_20"] - g["ma_20"].shift(1)
    
    return g

df = df.groupby("symbol", group_keys=False).apply(add_price_trend_features)

# ==========================
# 3. 변동성 지표
# ==========================
def add_volatility_features(g):
    g = g.copy()
    
    # True Range (TR)
    tr1 = g["high"] - g["low"]
    tr2 = (g["high"] - g["prdy_close"]).abs()
    tr3 = (g["low"] - g["prdy_close"]).abs()
    g["tr"] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    
    # ATR(5), ATR(14)
    g["atr_5"] = g["tr"].rolling(5).mean()
    g["atr_14"] = g["tr"].rolling(14).mean()
    
    # 로그수익률
    g["ln_return"] = np.log(g["close"] / g["prdy_close"])
    
    # RV(10), RV(20)
    g["rv_10"] = g["ln_return"].rolling(10).std()
    g["rv_20"] = g["ln_return"].rolling(20).std()
    
    # Range% (당일 고저차 / 종가)
    g["range_pct"] = (g["high"] - g["low"]) / g["close"]
    
    # Gap% (시가 대비 전일 종가)
    g["gap_pct"] = (g["open"] - g["prdy_close"]) / g["prdy_close"]
    
    return g

df = df.groupby("symbol", group_keys=False).apply(add_volatility_features)


FileNotFoundError: [Errno 2] No such file or directory: '/data/proc/daily_raw_clean.csv'

In [ ]:
display(df.head(3))
display(df.tail(3))

In [ ]:
# ==========================
# 4. 결과 저장
# ==========================
save_path = "/data/proc/daily_with_indicators.csv"
df.to_csv(save_path, index=False)

print("✅ 보조지표 추가 완료:", save_path)
print("샘플 확인:")
print(df[df["symbol"] == df["symbol"].iloc[0]].tail(3))